In [ ]:
from os import path

import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import KernelDensity

# gala
import gala.coordinates as gc
import gala.integrate as gi
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData

In [ ]:
mw_pot = gp.MilkyWayPotential()
gc_frame = coord.Galactocentric(galcen_distance=8.1*u.kpc, # GRAVITY
                                z_sun=27*u.pc) 

In [ ]:
suh = at.Table.read('/Users/apricewhelan/data/GaiaDR2/DR2_RRL_nocontam.csv.gz', 
                    format='ascii.csv')
rrl = GaiaData(suh)

In [ ]:
baum_tbl = at.Table.read(path.expanduser('~/data/Misc/Baumgardt-globclust.fits'))
vasi_tbl = at.Table.read(path.expanduser('~/data/Misc/Vasiliev-globclust.txt'), 
                         format='ascii.fixed_width')

In [ ]:
ix1 = []
ix2 = []
for i, row in enumerate(baum_tbl):
    name = row['Name']
    
    # HACKS:
    if 'Ter' in name:
        name = name.replace('Ter', 'Terzan')
    elif 'Lil' in name:
        name = name.replace('Lil', 'Liller')
    elif name == 'ESO 452-SC11':
        name = 'ESO 452-11'
    
    for j, vrow in enumerate(vasi_tbl):
        if name in vrow['Name']:
            ix1.append(i)
            ix2.append(j)
            break
    else:
        print(name + ' not found')
            
ix1 = np.array(ix1)
ix2 = np.array(ix2)

tbl = at.hstack((baum_tbl[ix1], vasi_tbl[ix2]))

tbl = tbl[np.isfinite(tbl['D']) & np.isfinite(tbl['Vlos'])]

In [ ]:
nsamples = 128
dist_samples = np.random.normal(tbl['D'], 0.1 * tbl['D'],
                                size=(nsamples, len(tbl)))
pm_samples = np.zeros((2, nsamples, len(tbl)))
for i in range(len(tbl)):
    row = tbl[i]
    x = [row['PMRA'], row['PMDEC']]
    C = np.diag([row['ePMRA'], row['ePMDEC']]) ** 2
    C[0, 1] = C[1, 0] = row['ePMRA'] * row['ePMDEC'] * row['corrPM']
    pm_samples[:, :, i] = np.random.multivariate_normal(x, C, size=nsamples).T
    
vr_samples = np.random.normal(tbl['Vlos'], tbl['eVlos'],
                              size=(nsamples, len(tbl)))

ra_samples = np.repeat(tbl['RA'][None], nsamples, axis=0)
dec_samples = np.repeat(tbl['DEC'][None], nsamples, axis=0)

In [ ]:
c = coord.SkyCoord(ra=ra_samples*u.deg,
                   dec=dec_samples*u.deg,
                   distance=dist_samples*u.kpc,
                   pm_ra_cosdec=pm_samples[0]*u.mas/u.yr,
                   pm_dec=pm_samples[1]*u.mas/u.yr,
                   radial_velocity=vr_samples*u.km/u.s)
c = c.T

w0 = gd.PhaseSpacePosition(c.transform_to(gc_frame).cartesian)

From GC-Orbits-Monte-Carlo.ipynb:

In [ ]:
df = gd.FardalStreamDF()

In [ ]:
names = ['NGC 6809', 'NGC 288', 'NGC 5897', 'NGC 6362', 'IC 4499', 
         'NGC 7099', 'Pal 13', 'NGC 6144', 'NGC 6101', 'NGC 1904']

In [ ]:
for name in names:
    row = tbl[tbl['Name_1'] == name]
    this_w0 = w0[tbl['Name_1'] == name][0]
    
    # Run mock stream models from samples from the orbit error distribution:
    mass = row['mass'][0]*u.Msun
    prog_pot = gp.PlummerPotential(m=mass, 
                                   b=row['r_hm'][0] / 1.3 * u.pc,
                                   units=galactic)
    
    gen = gd.MockStreamGenerator(df, mw_pot,
                                 progenitor_potential=prog_pot)
    
    streams = []
    for i in tqdm(range(32)):
        stream, _ = gen.run(this_w0[i], mass, release_every=8,
                            dt=-1*u.Myr, n_steps=2000)

        streams.append(stream)
        
    # Now transform to sky coordinates, centered on the cluster:
    fr = coord.SkyOffsetFrame(origin=coord.ICRS(ra=row['RA'][0]*u.deg, 
                                                dec=row['DEC'][0]*u.deg))

    all_sky = []
    for stream in streams:
        stream_c = stream.to_coord_frame(fr, galactocentric_frame=gc_frame)
        all_sky.append(np.stack((stream_c.lon.wrap_at(180*u.deg).degree, 
                                 stream_c.lat.degree)).T)

    all_sky = np.vstack(all_sky)
    all_sky = all_sky[np.sqrt(all_sky[:, 0]**2 + all_sky[:, 1]**2) > 0.25]
    
    # Predict where we expect to see tidal debris:
    kde = KernelDensity(bandwidth=0.2)
    _ = kde.fit(all_sky)

    grid = np.arange(-10, 10+1e-3, 0.25)
    xgrid, ygrid = np.meshgrid(grid, grid)
    X_grid = np.stack((xgrid.ravel(), ygrid.ravel())).T

    H = np.exp(kde.score_samples(X_grid))
    H = H.reshape(grid.size, grid.size)
    
    # --- plot
    
    fig, ax = plt.subplots(1, 1, figsize=(6.2, 6))
    ax.pcolormesh(xgrid, ygrid, H)

    ax.set_xlabel(r'$\phi_1$')
    ax.set_ylabel(r'$\phi_2$')

    ax.set_title('Predicted stream ({})'.format(row['Name_1'][0]), 
                 fontsize=16)

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_ticks(np.arange(-10, 10+1e-3, 5))

    fig.set_facecolor('w')

    fig.tight_layout()
    fig.savefig('../plots/predicted-debris-{}.png'.format(row['Name_1'][0].replace(' ', '_')),
                dpi=250)

---

In [ ]:
# row = tbl[tbl['Name_1'] == 'NGC 5897']
row = tbl[tbl['Name_1'] == 'Pal 13']

# Now transform to sky coordinates, centered on the cluster:
fr = coord.SkyOffsetFrame(origin=coord.ICRS(ra=row['RA'][0]*u.deg, 
                                            dec=row['DEC'][0]*u.deg))

In [ ]:
rrl_c = rrl.get_skycoord(distance=rrl.D_kpc*u.kpc)

In [ ]:
rrl_c_fr = rrl_c.transform_to(fr)
sky_mask = (np.abs(rrl_c_fr.lon) < 10*u.deg) & (np.abs(rrl_c_fr.lat) < 10*u.deg)
pm_mask = np.sqrt((rrl_c.pm_ra_cosdec.value - (row['PMRA'][0]))**2 + 
                  (rrl_c.pm_dec.value - row['PMDEC'][0])**2) < 1.5
dist_mask = (np.abs(rrl_c.distance - row['dist'][0]*u.kpc) < 5*u.kpc)
# dist_mask = (np.abs(rrl_c.distance - 10*u.kpc) < 2*u.kpc)
mask = sky_mask & dist_mask & pm_mask

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

ax = axes[0]
ax.plot(rrl_c_fr.lon.degree[mask], 
        rrl_c_fr.lat.degree[mask],
        ls='', marker='o', mew=0, ms=2., alpha=0.5)
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax = axes[1]
ax.plot(rrl_c.pm_ra_cosdec.value[mask], 
        rrl_c.pm_dec.value[mask],
        ls='', marker='o', mew=0, ms=2., alpha=0.5)
# ax.scatter(row['PMRA'], row['PMDEC'])